In [36]:
# this leverages python to auto create and import data into postgreSQL databases

import psycopg2 as psy
import pandas as pd
import numpy as np
import os

In [30]:
conn = psy.connect("host=localhost dbname=Mt_View_Census user=postgres password=Recol123")
cur = conn.cursor()


In [38]:
path = r"C:\Data\Waste_Intervention\Census_Tables\Cleaned"
fList = os.listdir(path)

In [40]:
dfs = {}

for doc in fList:
    i = "{}\\{}".format(path, doc)
    j = doc.split("_")[-1][:-4]
    
    dfs[j] = pd.read_csv(i, na_values=("-"))
    
    df = pd.read_csv(i, na_values=("-"))
    #print dfs  
        
    df = df.dropna()#[df.select_dtypes(['object']).eq('-').any(1)]
    print df.head(10)
    df.to_csv(i, index=False)
    


           Id2  med_age
0  60855001001     27.2
1  60855001002     33.7
2  60855001003     26.9
3  60855001004     34.1
4  60855002001     30.6
5  60855002002     36.9
6  60855002003     36.9
7  60855002004     29.2
8  60855003001     37.3
9  60855003002     35.4
           Id2  White  AfAm  NatAm  Asian  Hawaiin  Other  Inter
0  60855001001    310     0      0    112        0    500     10
1  60855001002    397    18     12    727        0    717     65
2  60855001003    731     0      0    403        0   1418     68
3  60855001004    755   150     22    559        0    756     94
4  60855002001    796    43     50    212       63    154    250
5  60855002002    337   118     12     46        0    123     58
6  60855002003    812    83      0    250        0    376    190
7  60855002004    895    88     29    629        0    602     53
8  60855003001    265    65      7     54        0     35    127
9  60855003002   1865   215     20    726        0    202    209
           Id2  Total

In [41]:
# make a list of all the csvs to bring into pgAdmin
path = r"C:\Data\Waste_Intervention\Census_Tables\Cleaned"
fList = os.listdir(path)

# create tables in pandas to get column names for each table
# use these values to create tables with psycopg2
for doc in fList:
    pathCSV = "{}\\{}".format(path, doc)
    tableName = doc.split("_")[-1][:-4] # this strips the descriptive part of the file path
                                        # for the table name
    table = pd.read_csv(pathCSV)
    
    pgCols = [] # this will hold column names and data types
    types=["BIGINT Primary Key"]
    cols = table.columns
    colsLen = len(table.columns)
    x = 1
    while x < colsLen:
        types.append("numeric")
        x += 1
    pgTups = zip(cols, types)
    pgCols.append(', '.join([str(lst[0]) + " " + str(lst[1]) for lst in pgTups]))
    # pgcols is now in the form column1 datatype, column2 2 datatype
            
    create_statement =  'CREATE TABLE ' + tableName + '(' + pgCols[0] + ')'
    # statement used in execute

    cur.execute(create_statement)
    
    # tale is created. now use same file in csv_read to copy data into table
    #cur.seek(0)
    
    
    copy_statement = "COPY " + tableName + " FROM STDIN CSV HEADER DELIMITER AS ','"
    print copy_statement
    with open(pathCSV, 'r') as f:
        cur.copy_expert(copy_statement, f)
    conn.commit()
    f.close()

COPY age FROM STDIN CSV HEADER DELIMITER AS ','
COPY race FROM STDIN CSV HEADER DELIMITER AS ','
COPY hhPop FROM STDIN CSV HEADER DELIMITER AS ','
COPY hhSize FROM STDIN CSV HEADER DELIMITER AS ','
COPY medHHInc FROM STDIN CSV HEADER DELIMITER AS ','
COPY vacant FROM STDIN CSV HEADER DELIMITER AS ','
COPY ownRentHH FROM STDIN CSV HEADER DELIMITER AS ','
COPY avHHsize FROM STDIN CSV HEADER DELIMITER AS ','
COPY housingDensity FROM STDIN CSV HEADER DELIMITER AS ','
COPY houseYear FROM STDIN CSV HEADER DELIMITER AS ','
COPY numBeds FROM STDIN CSV HEADER DELIMITER AS ','
COPY rentBrackets FROM STDIN CSV HEADER DELIMITER AS ','
COPY bachelors FROM STDIN CSV HEADER DELIMITER AS ','
COPY totalPop FROM STDIN CSV HEADER DELIMITER AS ','
